In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np

import copy
import cv2
from neural_style import stylize_img
from scipy.misc import imsave
import cvbase as cvb
import glob

0.4.0
0.2.1


In [2]:
# Directory with video frames and flo files. (Created from setup.ipynb)
videoDir = "test-videos/iguana_pan"

In [3]:
#get all image/flo names
images = sorted(glob.glob( videoDir + "/frame*.png"))
flos = sorted(glob.glob(videoDir + "/*.flo"))

In [4]:
#Load images as tensors, flow warping can be done on CPU because it isn't computationally intensive

device = "cpu"
imsize = (480, 640)
loader = transforms.Compose([
    transforms.Resize(imsize),# scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor
unloader = transforms.ToPILImage()

def im_loader(image_name, flipped = False):
    image = Image.open(image_name)
    
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [5]:
#Warps an input image by the optical flow

def flow_warp(original, flostr):
    original = (original.view(3,480,640).numpy()).clip(0,1)
    original = np.swapaxes(original,2,0)
    original = np.swapaxes(original,1,0)
    
    flo = cvb.read_flow(flostr)
    flo = np.swapaxes(flo,2,0)
    flo = np.swapaxes(flo,1,2)
    _, h, w = flo.shape
    flow_map = np.zeros(flo.shape,dtype=np.float32)
    
    for y in range(h):
        flow_map[1,y,:] = float(y) - flo[1,y,:]
    for x in range(w):
        flow_map[0,:,x] = float(x) - flo[0,:,x]
    warped = np.zeros_like(original)
    
    warped[:,:,0] = cv2.remap(original[:,:,0],flow_map[0],flow_map[1],
                   interpolation=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    warped[:,:,1] = cv2.remap(original[:,:,1],flow_map[0],flow_map[1],
                   interpolation=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    warped[:,:,2] = cv2.remap(original[:,:,2],flow_map[0],flow_map[1],
                   interpolation=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    
    return warped.clip(0,1)

In [6]:
# Loads im and flo file into warp function

def get_warped(im1,flo):
    flo_im = cvb.flow2rgb(cvb.read_flow(flo))
    im1 = im_loader(im1)
    warped = flow_warp(im1,flo)
    return warped

In [8]:
#Change the tag and style image to what you like.
style_img = 'style/kazao.jpg'
style_tag = 'kazaotest'
def style_name(img_name):
    img_name = img_name.split("frame")
    return img_name[-2] + style_tag + img_name[-1]

In [9]:
#Saves the frame warped by the optical flow in temp.png (Temp because it's continually overwritten)
warp = 'temp.png'

#Run the stylization algorithm for many steps on the first image
stylize_img(images[0],style_img,images[0],style_name(images[0]), num_steps = 700)
warped = get_warped(style_name(images[0]),flos[0])
imsave(warp, warped)

#For every image after, it only needs to be ran for a fraction of the iterations
for im, flo in zip(images[1:],flos[1:]):
    stylize_img(im, style_img, warp, style_name(im), num_steps = 200)
    warped = get_warped(style_name(im),flo)
    imsave("temp.png", warped)

Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 0.000034 Content Loss: 44.684731
()
run [100]:
Style Loss : 0.000009 Content Loss: 40.283005
()
run [150]:
Style Loss : 0.000005 Content Loss: 36.515656
()
run [200]:
Style Loss : 0.000004 Content Loss: 34.560608
()


KeyboardInterrupt: 